<a href="https://colab.research.google.com/github/Krati012/IDP-2020/blob/master/CNN_2019data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls "/content/gdrive/My Drive/CTW_2019"

CTW2019_Dataset_h5.zip	h_Estimated_CTW_Train.h5  test.out
data2019.h5		r_Position_CTW_Train.h5
data_feat.h5		SNR_CTW_Train.h5


In [0]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
import numpy as np # linear algebra
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, ReLU, Input, Dropout
import numpy as np
import os
import h5py
import pandas as pd
import time
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
#getting and processing data
#h_Estimated   ---> shape = (2, 924, 16, 17486)
#r_postition ---> shape = (3, 17486)
#SNR_Est    ---> shape = (16, 17486)


batch_size = 30
epochs = 50
data_augmentation = False
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cnn_training.h5'


In [0]:
f = h5py.File('/content/gdrive/My Drive/CTW_2019/h_Estimated_CTW_Train.h5','r')
H_re = f['h_Estimated'][0,:]
H_im = f['h_Estimated'][1,:]
print(H_re.shape)
H = f['h_Estimated']

(924, 16, 17486)


In [0]:
f = h5py.File('/content/gdrive/My Drive/CTW_2019/r_Position_CTW_Train.h5','r')
pos = f['r_Position']
f = h5py.File('/content/gdrive/My Drive/CTW_2019/SNR_CTW_Train.h5','r')
SNR = f['SNR_Est']

In [0]:
'''#H_new = keras.backend.transpose(H)
#print(H_new.shape)
#num_classes = 1
y_train = keras.utils.to_categorical(pos)
print(y_train.shape)'''
import tensorflow as tf

def tens(x):
  y = tf.convert_to_tensor(x)
  return y

if train_on_gpu:
            H, pos = H.cuda(), pos.cuda()

H_train = tens(H)
print(H_train.shape)
pos_train = tens(pos)

AttributeError: ignored

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(H_train,pos_train, test_size=0.2, random_state=42, shuffle=True)
#print(H_new.shape,pos.shape)

print(X_train.shape,X_val.shape)
print(y_train.shape,y_val.shape)

print(len(H),len(pos))

In [0]:

model = Sequential()

model.add(Conv2D(32, (3,3), padding="same"))
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_size=4))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3,3), padding="same"))
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_size=4))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(3))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(learning_rate=0.001, decay=1e-6)

print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_val, y_val),
              shuffle=True)
    

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(X_val, y_val, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

NameError: ignored